In [14]:
# 導入模組
import pandas as pd
import numpy as np
import json
import math

from flask import Flask, request, abort

from flask_ngrok import run_with_ngrok

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)


In [15]:
app = Flask(__name__, static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

line_bot_api = LineBotApi("4zAj0CZfakjEn86lQkcWWoS6QtzsZxlnDKKWRJrlF2KbzzQUFdDAILa+V0W+zxz+uvA4k8n0+rZCI+tjB8/G5cI03/lJKDRsd+HnAaXKGwJon4zyt9X08nhrPflSoWFoKEZFfndot6SGkiPGI4G9iwdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("bdd1c3f1804aefb0d02a68fb42b4a342")

In [16]:
df = pd.read_csv('/Users/franklin/Desktop/Final_Project/new_all_data.csv')

In [17]:
@app.route("/", methods=['POST'])

def callback():
    
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)
    
    # 記錄用戶log
    f = open("/Users/franklin/Desktop/Final_Project/bot_event.txt", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [18]:
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton, TextSendMessage
)

# 創建QuickReplyButton 
## 點擊後，跳出地理位置
locationQRB = QuickReplyButton(
    action=LocationAction(label="地理位置")
)
## 設計QuickReplyButton的List
quickReplyList = QuickReply(
    items = [locationQRB]
)
## 將quickReplyList 塞入TextSendMessage 中 
quick_reply_text_send_message = TextSendMessage(text='請提供您的位置資訊:', quick_reply=quickReplyList)

In [19]:
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage, LocationMessage
)

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
  "@現居地評分":quick_reply_text_send_message,
  "@我要找機能":quick_reply_text_send_message,
  "@more":"hello",
}

In [20]:
from linebot.models.events import (
    FollowEvent
)
from linebot.models import(
    TextSendMessage, ImageSendMessage
)
import requests

# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("/Users/franklin/Desktop/Final_Project/log.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')
    
    # 回覆文字消息與圖片消息(歡迎詞)
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage('本系統將為您提供生活機能評分功能,無論您是上班族/小家庭/銀髮族...,可以透過本系統查看居住環境的周遭機能,請透過下方圖文選單點選功能')]
    )

In [21]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):

    if '@' in event.message.text:
      line_bot_api.reply_message(
      event.reply_token,
      template_message_dict.get(event.message.text)
      )
    elif '你選了' in event.message.text:
      line_bot_api.reply_message(
      event.reply_token,
      TextSendMessage(text="測試")
      )
    else:
      line_bot_api.reply_message(
      event.reply_token,
      TextSendMessage(text="查無此關鍵字，請透過圖文選單點選功能進行使用")
      )


In [22]:
def Distance(B,user_lati,user_longi):
  # B -> [25.123, 121.123]
  
  [a, b] = user_lati,user_longi
  [c, d] = eval(B)
  
  import math

  R = 6373.0  # radius of the Earth

  # coordinates
  lat1 = math.radians(a)
  lon1 = math.radians(b)
  lat2 = math.radians(c)
  lon2 = math.radians(d)

  # change in coordinates
  dlon = lon2 - lon1
  dlat = lat2 - lat1

  # Haversine formula
  a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
  c = 2 * math.asin(math.sqrt(a))
  distance = R * c

  return distance

In [23]:
def surroundings(User):
  sur = User[User['distance'] <= 1]
  sur["adj_score"] = sur["rating"]*sur["senti_score"]
  uniq = []
  all_type = []
  for i in sur['types']:
    all_type.extend(eval(i))
  uniq = sorted(list(zip(set(all_type), list(all_type.count(j) for j in set(all_type)))), key= lambda x: x[-1], reverse=True)
  return "加權評分: "+round(sur["adj_score"].mean(),3)+" (googles評分*SnowNLP分數)/n"+str(uniq[:5])

In [24]:
@handler.add(MessageEvent, message=LocationMessage)
def handle_message(event):
  print(event.message)
  print(type(event.message))
  loca = event.message

  user_lati = loca.latitude
  user_longi = loca.longitude
  user_add = loca.address
  user_id = loca.id
  print(user_lati , user_longi , user_add , user_id)
  # print(Distance(B,user_lati,user_longi))
  
  df['distance'] = df['location'].apply(lambda x : Distance(x,user_lati,user_longi))
  User = df.sort_values(by=['distance']).reset_index(drop=True)
  print(surroundings(User))
  
  line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=surroundings(User))
    )


In [25]:
from linebot.models import(
    MessageEvent,ImageMessage, TextSendMessage
)

@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Image has Upload'+ ' ' + event.message.id))
    message_content = line_bot_api.get_message_content(event.message.id)
    with open(event.message.id+'.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [ ]:
app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/threading.py", line 1177, in run
    self.function(*self.args, **self.kwargs)
  File "/usr/local/lib/python3.7/site-packages/flask_ngrok.py", line 70, in start_ngrok
    ngrok_address = _run_ngrok()
  File "/usr/local/lib/python3.7/site-packages/flask_ngrok.py", line 31, in _run_ngrok
    ngrok = subprocess.Popen([executable, 'http', '5000'])
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/subprocess.py", line 800, in __init__
    restore_signals, start_new_session)
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/subprocess.py", 